<a href="https://colab.research.google.com/github/anderson-ferreira-83/Data_Science_Repo_anderson83/blob/main/1_Alura_Voz/Week_4_model_validation/p4_Optimization_for_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 4 - Optimization

In [ ]:
#
import os
import sys

In [ ]:
#
import pandas as pd

For the final presentation of our analysis for Alura Voice, we need to define the best model and use the best hyperparameters in it.

### Importing libraries

For the application we will use `pandas`, `seaborn`, `sklearn`, `imblearn` and `sys`.

In [ ]:
# Link to access the functions of model
str_utils = '1DJEF0jli6eQixbcz-ARBX7X5d9ojoP4J'

In [ ]:
# Downloading file to current GoogleColab directory 'utils.py'
!gdown --id $str_utils

In [ ]:
#
from utils import plot_countplot,plot_matrix_confusion, compare_models_metrics

In [ ]:
#
import numpy as np
import pandas as pd
#
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#
SEED = 42

## Optimizing the model

Knowing that it is necessary to choose the best developed model, we will analyze the result obtained in the [previous study](https://github.com/sthemonica/challenge_dados_1/blob/bruno/3-Modelos%20de%20ML/modelos.ipynb), ordered by the Recall value:

<h3>Table of models ordered by Recall</h3>

|    | Model          | Training Accuracy | Test Accuracy | Precision | Recall | F1-Score |
|---:|:---------------|------------------:|--------------:|----------:|-------:|---------:|
|  0 | random forest  |              0.81 |          0.80 |      0.77 |   0.85 |     0.81 |
|  1 | decision tree  |              0.81 |          0.80 |      0.78 |   0.85 |     0.81 |
|  2 | svc            |              0.75 |          0.75 |      0.73 |   0.78 |     0.75 |


***Recall*** is a metric that evaluates whether positive predictions were given correctly, that is, it evaluates the number of True Positives (True Pos) and False Negatives (False Neg). Therefore, the higher the Recall, the more correct the model obtained.

Therefore, we will use the model that obtained the best Recall, therefore, Random Forest.

Having chosen the best model, we will use [GridSearchCV](https://cursos.alura.com.br/course/machine-learning-otimizacao-de-modelos-atraves-de-hiperparametros/task/48715) to optimize it.

Now, let's read the dataset worked on in the previous notebook `models.ipynb` through the pandas `read_json` method:

In [ ]:
# Link to access the shared database file 'Telco-Customer-Churn-balancing.json'
str_data_telco_cust_churn_balanc_file = '1pwoPlr7KE5mIuTae54mJYxVfe9nC-BzQ'

In [ ]:
# Downloading file to current GoogleColab directory 'Telco-Customer-Churn-balancing.json'
!gdown --id $str_data_telco_cust_churn_balanc_file

In [ ]:
data = pd.read_json('Telco-Customer-Churn-balancing.json')
data.head()

To optimize the model, we first need to create the training and testing set. In this way, we divide the dataset into input (X) and output (y), or target.

In [ ]:
X = data.drop(['Churn'], axis=1)
y = data['Churn']

And so we can separate the data into training and testing with `train_test_split`.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

It is necessary to define the hyperparameters to be optimized by GridSearchCV, which are:

* **`n_estimators`**: number of decision trees;
* **`criterion`**: decision criterion;
* **`max_features`**: maximum number of variables to be used when searching for the best division;
* **`max_depth`**: maximum depth of the decision tree;
* **`min_samples_split`**: minimum number of samples required for a division to be made;
* **`min_samples_leaf`**: minimum number of samples required for a leaf to be formed;
* **`bootstrap`**: indicates whether data resampling was performed when creating the decision tree models.

In [ ]:
n_estimators = np.arange(100, 200, step=20)
criterion = ["gini", "entropy"]
max_features = ["auto", "log2"]
max_depth = list(np.arange(2, 10, step=2))
min_samples_split = np.arange(2, 10, step=2)
min_samples_leaf = [2, 4]
bootstrap = [True, False]

parameters = {
    "n_estimators": n_estimators,
    "criterion": criterion,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

Now, we can use the GridSearchCV optimizer to improve the *Recall* metric. To do this, we created a GridSearchCV object and passed our model, the hyperparameters defined previously, and the metric to be optimized, *Recall*, as parameters. We also performed the adjustment using the `fit()` method.

To learn more about GridSearchCV and its parameters, you can consult the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
clf = GridSearchCV(RandomForestClassifier(random_state=SEED), parameters, cv=3, n_jobs=-1, scoring="recall")
clf.fit(x_train, y_train)  # ajuste com os dados de treino

After the adjustment, we were able to analyze the results obtained:

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by="rank_test_score").head()

To obtain the best results from the optimizer we use the **`best_params_`** command:

In [ ]:
clf.best_params_

With this, we were able to create a Random Forest estimator with the best hyperparameters:

In [ ]:
rforest = RandomForestClassifier(**clf.best_params_, random_state=SEED)

In [ ]:
model = rforest.fit(x_train, y_train)  # treinamento com os dados de treino
y_pred = rforest.predict(x_test)  # realizando as predições

We can analyze whether the model actually obtained better results in the *Recall* metric using the `plot_matriz_confusao` function:

In [ ]:
labels = ["True Neg","False Pos","False Neg","True Pos"]
categories = ["Não Churn", "Churn"]
plot_matrix_confusion(y_test,
                      y_pred,
                      group_names=labels,
                      categories=categories,
                      figsize=(8, 6),
                      title="Matriz de confusão para o classificador Random Forest")

In [ ]:
print(f"Training accuracy: {rforest.score(x_train, y_train) * 100:.2f}%")  # Checking training accuracy

It can be seen that recall and other metrics increased with the optimized model.

<h4>Before optimization</h4>

|    | Model          | Training Accuracy | Test Accuracy | Precision | Recall | F1-Score |
|---:|:---------------|------------------:|--------------:|----------:|-------:|---------:|
|  0 | random forest  |              0.81 |          0.80 |      0.77 |   0.85 |     0.81 |




<h4>After optimization</h4>

|    | Model          | Training Accuracy | Test Accuracy | Precision | Recall | F1-Score |
|---:|:---------------|------------------:|--------------:|----------:|-------:|---------:|
|  0 | random forest  |              0.85 |          0.82 |      0.79 |   0.86 |     0.83 |


This way, we were able to create a well-optimized model and present it to Alura Vocing.